In [1]:
from python_workflow_definition.aiida import write_workflow_json
from python_workflow_definition.jobflow import load_workflow_json
from jobflow.managers.local import run_locally

from aiida_workgraph import WorkGraph, task
from aiida import load_profile
load_profile()

workflow_json_filename =  "aiida_to_jobflow_simple.json"

/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
/srv/conda/envs/notebook/lib/python3.12/site-packages/paramiko/transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
from simple_workflow import (
    add_x_and_y as _add_x_and_y, 
    add_x_and_y_and_z as _add_x_and_y_and_z,
)

In [3]:
@task.pythonjob()
def pickle_node(value):
    """Handle data nodes"""
    return value

In [4]:
add_x_and_y = task.pythonjob(outputs=["x", "y", "z"])(_add_x_and_y)
add_x_and_y_and_z = task.pythonjob()(_add_x_and_y_and_z)

In [5]:
# TODO: Create inputs rather than tasks out of data nodes
wg = WorkGraph("wg-simple")

In [6]:
helper_task1 = wg.add_task(pickle_node, name="x", value=1)
helper_task2 = wg.add_task(pickle_node, name="y", value=2)

In [7]:
add_x_and_y_task = wg.add_task(
    add_x_and_y,
    name="add_x_and_y",
    x=helper_task1.outputs.result,
    y=helper_task2.outputs.result,
)

In [8]:
add_x_and_y_and_z_task = wg.add_task(
    add_x_and_y_and_z,
    name="add_x_and_y_and_z",
    x=add_x_and_y_task.outputs.x,
    y=add_x_and_y_task.outputs.y,
    z=add_x_and_y_task.outputs.z,
)

In [9]:
write_workflow_json(wg=wg, file_name=workflow_json_filename)

{'nodes': {'0': 1,
  '1': 2,
  '2': 'simple_workflow.add_x_and_y',
  '3': 'simple_workflow.add_x_and_y_and_z'},
 'edges': [{'target': 2,
   'targetHandle': 'x',
   'source': 0,
   'sourceHandle': None},
  {'target': 2, 'targetHandle': 'y', 'source': 1, 'sourceHandle': None},
  {'target': 3, 'targetHandle': 'x', 'source': 2, 'sourceHandle': 'x'},
  {'target': 3, 'targetHandle': 'y', 'source': 2, 'sourceHandle': 'y'},
  {'target': 3, 'targetHandle': 'z', 'source': 2, 'sourceHandle': 'z'}]}

In [10]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [11]:
result = run_locally(flow)
result

2025-03-21 20:45:40,878 INFO Started executing jobs locally
2025-03-21 20:45:41,115 INFO Starting job - add_x_and_y (ef28440e-7d32-49d3-85f5-121da0fdf65f)
2025-03-21 20:45:41,117 INFO Finished job - add_x_and_y (ef28440e-7d32-49d3-85f5-121da0fdf65f)
2025-03-21 20:45:41,118 INFO Starting job - add_x_and_y_and_z (c0da46a8-a430-4d88-adb6-7d5b36b8fe52)
2025-03-21 20:45:41,119 INFO Finished job - add_x_and_y_and_z (c0da46a8-a430-4d88-adb6-7d5b36b8fe52)
2025-03-21 20:45:41,120 INFO Finished executing jobs locally


{'ef28440e-7d32-49d3-85f5-121da0fdf65f': {1: Response(output={'x': 1, 'y': 2, 'z': 3}, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))},
 'c0da46a8-a430-4d88-adb6-7d5b36b8fe52': {1: Response(output=6, detour=None, addition=None, replace=None, stored_data=None, stop_children=False, stop_jobflow=False, job_dir=PosixPath('/home/jovyan'))}}